In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                product_id = resp.split('window.productId = ')[1].split(';')[0]
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//div[@class="product-details"]/div[@class="row"]/div[@class="col-12"]/div/h1/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="main-img"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_li = html.xpath('//div[@id="product-description-tab"]/div[1]/ul/li')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Product_Id': product_id,
                                         'Vehicle': '',
                                         'OE': '',
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for li in list_li:
                    list_span = li.xpath('./span')[1:]
                    text_span = ''
                    for span in list_span:
                        text_span += span.xpath('./text()')[0].strip() + '\n'
                    df_temp.loc[0, li.xpath('./span')[0].xpath('./text()')[0].strip()[:-1]] = text_span.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1

[ok] - https://www.1aauto.com/ford-lincoln-rear-driver-and-passenger-side-2-piece-performance-brake-caliper-set-trq-performance-cla49883/i/1abcs00516
[尝试次数：1] - [剩余数量：0] - [当前时间：15:59:50]

输出ing...

Done ~
